PyTorch神经网络基础

1.自定义块

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F


# 自定义多层感知机
class MLP(nn.Module):
    def __init__(self):
        super().__init__()  # 调用父类的__init__函数
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    # 正向传播过程
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
                # 输出层  <-  隐藏层   <-  输入X

# 实例化多层感知机的层，然后在每次调用正向传播函数调用这些层
net = MLP()
X = torch.rand(2, 20)
net(X)

顺序块

该模型类的构造函数接受任意数量的模块作为参数，并将它们存储在自身的模块字典_modules中

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F


class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            print(block)
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256),nn.ReLU(),nn.Linear(256,10))
X = torch.rand(2,20)
net(X)

正向传播

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F


# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight + 1))
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
X = torch.rand(2,20)
net(X)

混合组合块

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F


# 混合代培各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)

    def forward(self, X):
        return self.linear(self.net(X))

chimear = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
X = torch.rand(2,20)
chimear(X)

嵌套块

In [ ]:
import torch
from torch import nn


net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand(size=(2,4))


# block1包含两个线性层和两个ReLU激活函数
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())


# 用block2返回整个net
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())
    return net


rgnet = nn.Sequential(block2(), nn.Linear(4,1))
print(rgnet(X))
print(rgnet)